# Laboratorio 04 --> Clasificación de Malware

Universidad del Valle de Guatemala

Luis Pedro Cuéllar - 18220


### Parte I --> Análisis de un Malware


In [1]:
import os
import glob
import pefile
import hashlib
import numpy as np
import pandas as pd

In [6]:
PATH = "MALWR2"

all_files = glob.glob("{}/*".format(PATH))
all_files

['MALWR2/sample_qwrty_dk2', 'MALWR2/sample_vg655_25th.exe']


#### **Análisis Estático**


In [19]:
df = []

for i in all_files:
    dll_calls = []
    api_calls = []
    
    pe = pefile.PE(i)
    for entry in pe.DIRECTORY_ENTRY_IMPORT:
        dll_calls.append(entry.dll.decode("utf-8"))
        
        for call in entry.imports:
            api_calls.append(call.name.decode("utf-8"))
    
    df.append(dll_calls)
    df.append(api_calls)
    
    print("\n")
    print("FILE NAME :\n\t {}".format(i))
    print("DLL CALLS :\n\t {}".format(dll_calls))
    print("API_CALLS :\n\t {}".format(api_calls))



FILE NAME :
	 MALWR2/sample_qwrty_dk2
DLL CALLS :
	 ['KERNEL32.DLL', 'MSVCRT.dll', 'SHELL32.dll', 'USER32.dll', 'WS2_32.dll']
API_CALLS :
	 ['LoadLibraryA', 'ExitProcess', 'GetProcAddress', 'VirtualProtect', 'atol', 'SHChangeNotify', 'LoadStringA', 'closesocket']


FILE NAME :
	 MALWR2/sample_vg655_25th.exe
DLL CALLS :
	 ['KERNEL32.dll', 'USER32.dll', 'ADVAPI32.dll', 'MSVCRT.dll']
API_CALLS :
	 ['GetFileAttributesW', 'GetFileSizeEx', 'CreateFileA', 'InitializeCriticalSection', 'DeleteCriticalSection', 'ReadFile', 'GetFileSize', 'WriteFile', 'LeaveCriticalSection', 'EnterCriticalSection', 'SetFileAttributesW', 'SetCurrentDirectoryW', 'CreateDirectoryW', 'GetTempPathW', 'GetWindowsDirectoryW', 'GetFileAttributesA', 'SizeofResource', 'LockResource', 'LoadResource', 'MultiByteToWideChar', 'Sleep', 'OpenMutexA', 'GetFullPathNameA', 'CopyFileA', 'GetModuleFileNameA', 'VirtualAlloc', 'VirtualFree', 'FreeLibrary', 'HeapAlloc', 'GetProcessHeap', 'GetModuleHandleA', 'SetLastError', 'VirtualPro

**1. ¿Qué  diferencias  observa  entre  los  ejemplos?  ¿Existe  algún  indicio 
sospechoso en la cantidad de DLLs y las APIs llamadas?**

Entre los dos archivos que se extrajeron, se puede observar que la cantidad de llamadas en el segudo archivo aumenta significativamente. En el segundo archivo podemos observar llamas a API como la de abrir, escribir y cerrar un archivo. También, se obtiene varios path, crea archivos con el path, con el timestamp y otras llamadas a APIs.


In [29]:
df = []

for i in all_files:
    e_magic = []
    signature = []
    sections = []
    
    pe = pefile.PE(i)
    
    print("\nFILE NAME :\n\t {}".format(i))
    
    e_magic.append(pe.DOS_HEADER.e_magic)
    e_magic.append(hex(pe.DOS_HEADER.e_magic))
    
    print("E_MAGIC :\n\t {}".format(e_magic[0]))
    print("E_MAGIC (HEX):\n\t {}".format(e_magic[1]))
    
    signature.append(pe.NT_HEADERS.Signature)
    signature.append(hex(pe.NT_HEADERS.Signature))
    
    print("SIGNATIURE :\n\t {}".format(signature[0]))
    print("SIGNATIURE (HEX):\n\t {}".format(signature[1]))
    
    for section in pe.sections:
        sections.append(section.Name.decode("utf-8"))
        sections.append(hex(section.VirtualAddress))
        sections.append(hex(section.Misc_VirtualSize))
        sections.append(hex(section.SizeOfRawData))
        
        print(section.Name.decode('utf-8'))
        print("\tVirtual Address: " + hex(section.VirtualAddress))
        print("\tVirtual Size: " + hex(section.Misc_VirtualSize))
        print("\tRaw Size: " + hex(section.SizeOfRawData))
    
    df.append(e_magic)
    df.append(signature)
    df.append(sections)


FILE NAME :
	 MALWR2/sample_qwrty_dk2
E_MAGIC :
	 23117
E_MAGIC (HEX):
	 0x5a4d
SIGNATIURE :
	 17744
SIGNATIURE (HEX):
	 0x4550
UPX0    
	Virtual Address: 0x1000
	Virtual Size: 0x5000
	Raw Size: 0x0
UPX1    
	Virtual Address: 0x6000
	Virtual Size: 0x1000
	Raw Size: 0x1000
.rsrc   
	Virtual Address: 0x7000
	Virtual Size: 0x1000
	Raw Size: 0x200

FILE NAME :
	 MALWR2/sample_vg655_25th.exe
E_MAGIC :
	 23117
E_MAGIC (HEX):
	 0x5a4d
SIGNATIURE :
	 17744
SIGNATIURE (HEX):
	 0x4550
.text   
	Virtual Address: 0x1000
	Virtual Size: 0x69b0
	Raw Size: 0x7000
.rdata  
	Virtual Address: 0x8000
	Virtual Size: 0x5f70
	Raw Size: 0x6000
.data   
	Virtual Address: 0xe000
	Virtual Size: 0x1958
	Raw Size: 0x2000
.rsrc   
	Virtual Address: 0x10000
	Virtual Size: 0x349fa0
	Raw Size: 0x34a000


**2. ¿Qué significa que algunas secciones tengan como parte de su nombre “upx”?**

Significa que este paquete se encuentra enpaquetado.

In [31]:
os.system("upx -d {}".format(all_files[0])) 

                       Ultimate Packer for eXecutables
                          Copyright (C) 1996 - 2020
UPX 3.96        Markus Oberhumer, Laszlo Molnar & John Reiser   Jan 23rd 2020

        File size         Ratio      Format      Name
   --------------------   ------   -----------   -----------
      8192 <-      5632   68.75%    win32/pe     sample_qwrty_dk2

Unpacked 1 file.


0

In [33]:
df = []

for i in all_files:
    dll_calls = []
    api_calls = []
    
    pe = pefile.PE(i)
    for entry in pe.DIRECTORY_ENTRY_IMPORT:
        dll_calls.append(entry.dll.decode("utf-8"))
        
        for call in entry.imports:
            api_calls.append(call.name.decode("utf-8"))
    
    df.append(dll_calls)
    df.append(api_calls)
    
    print("\n")
    print("FILE NAME :\n\t {}".format(i))
    print("DLL CALLS :\n\t {}".format(dll_calls))
    print("API_CALLS :\n\t {}".format(api_calls))



FILE NAME :
	 MALWR2/sample_qwrty_dk2
DLL CALLS :
	 ['KERNEL32.DLL', 'MSVCRT.dll', 'SHELL32.dll', 'USER32.dll', 'WS2_32.dll']
API_CALLS :
	 ['CloseHandle', 'WaitForSingleObject', 'CreateEventA', 'ExitThread', 'Sleep', 'GetComputerNameA', 'CreatePipe', 'DisconnectNamedPipe', 'TerminateProcess', 'WaitForMultipleObjects', 'TerminateThread', 'CreateThread', 'CreateProcessA', 'DuplicateHandle', 'GetCurrentProcess', 'ReadFile', 'PeekNamedPipe', 'SetEvent', 'WriteFile', 'SetProcessPriorityBoost', 'SetThreadPriority', 'GetCurrentThread', 'SetPriorityClass', 'lstrcatA', 'lstrcpyA', 'GetEnvironmentVariableA', 'GetShortPathNameA', 'GetModuleFileNameA', 'GetStartupInfoA', 'GetModuleHandleA', '_controlfp', '_beginthread', '_strnicmp', 'sprintf', 'atol', 'strchr', 'free', 'malloc', '_exit', '_XcptFilter', 'exit', '_acmdln', '__getmainargs', '_initterm', '__setusermatherr', '_adjust_fdiv', '__p__commode', '__p__fmode', '__set_app_type', '_except_handler3', '_itoa', 'ShellExecuteExA', 'SHChangeNotif

In [32]:
df = []

for i in all_files:
    e_magic = []
    signature = []
    sections = []
    
    pe = pefile.PE(i)
    
    print("\nFILE NAME :\n\t {}".format(i))
    
    e_magic.append(pe.DOS_HEADER.e_magic)
    e_magic.append(hex(pe.DOS_HEADER.e_magic))
    
    print("E_MAGIC :\n\t {}".format(e_magic[0]))
    print("E_MAGIC (HEX):\n\t {}".format(e_magic[1]))
    
    signature.append(pe.NT_HEADERS.Signature)
    signature.append(hex(pe.NT_HEADERS.Signature))
    
    print("SIGNATIURE :\n\t {}".format(signature[0]))
    print("SIGNATIURE (HEX):\n\t {}".format(signature[1]))
    
    for section in pe.sections:
        sections.append(section.Name.decode("utf-8"))
        sections.append(hex(section.VirtualAddress))
        sections.append(hex(section.Misc_VirtualSize))
        sections.append(hex(section.SizeOfRawData))
        
        print(section.Name.decode('utf-8'))
        print("\tVirtual Address: " + hex(section.VirtualAddress))
        print("\tVirtual Size: " + hex(section.Misc_VirtualSize))
        print("\tRaw Size: " + hex(section.SizeOfRawData))
    
    df.append(e_magic)
    df.append(signature)
    df.append(sections)


FILE NAME :
	 MALWR2/sample_qwrty_dk2
E_MAGIC :
	 23117
E_MAGIC (HEX):
	 0x5a4d
SIGNATIURE :
	 17744
SIGNATIURE (HEX):
	 0x4550
.text   
	Virtual Address: 0x1000
	Virtual Size: 0xea6
	Raw Size: 0x1000
.rdata  
	Virtual Address: 0x2000
	Virtual Size: 0x67e
	Raw Size: 0x800
.data   
	Virtual Address: 0x3000
	Virtual Size: 0x628
	Raw Size: 0x200
.rsrc   
	Virtual Address: 0x4000
	Virtual Size: 0x80
	Raw Size: 0x200

FILE NAME :
	 MALWR2/sample_vg655_25th.exe
E_MAGIC :
	 23117
E_MAGIC (HEX):
	 0x5a4d
SIGNATIURE :
	 17744
SIGNATIURE (HEX):
	 0x4550
.text   
	Virtual Address: 0x1000
	Virtual Size: 0x69b0
	Raw Size: 0x7000
.rdata  
	Virtual Address: 0x8000
	Virtual Size: 0x5f70
	Raw Size: 0x6000
.data   
	Virtual Address: 0xe000
	Virtual Size: 0x1958
	Raw Size: 0x2000
.rsrc   
	Virtual Address: 0x10000
	Virtual Size: 0x349fa0
	Raw Size: 0x34a000
